# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [1]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='harrypotter'
)

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [2]:
from haystack.nodes import ElasticsearchRetriever
from haystack.nodes import FARMReader

retriever = ElasticsearchRetriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

And now we initialize our ODQA pipeline.

In [3]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [4]:
qa.run(query='What does McGonagall teach?')

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|██████████| 2/2 [00:13<00:00,  6.69s/ Batches]


{'query': 'What does McGonagall teach?',
 'no_ans_gap': 6.313351392745972,
 'answers': [<Answer {'answer': 'Transfiguration', 'type': 'extractive', 'score': 0.8863391876220703, 'context': "This transforming spell , was a spell that could turn Lovebirds into Love Notes, written messages to given to others as romantic gestures.\n\n\n==History==\nIn February 1989, Professor Minerva McGonagall decided to teach her Fifth year Transfiguration students at Hogwarts this spell, as part of the school's Valentine's Day celebrations.", 'offsets_in_document': [{'start': 232, 'end': 247}], 'offsets_in_context': [{'start': 232, 'end': 247}], 'document_ids': ['eaabd6415a0cb6fa96806df5272e4b10'], 'meta': {'url': 'https://harrypotter.fandom.com/wiki/Lovebird_to_Love_Note', 'name': 'Lovebird_to_Love_Note'}}>,
  <Answer {'answer': 'Transfiguration', 'type': 'extractive', 'score': 0.6055291891098022, 'context': ' the Dumbledore family. Around 30 years\xa0later, Dumbledore was\xa0a contributing columnist by

In [5]:
qa.run(query='What is Harry James Potter birthday?',
       params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 3}})

/Users/bpakra200/opt/anaconda3/envs/drexel/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
Inferencing Samples: 100%|██████████| 1/1 [00:05<00:00,  5.39s/ Batches]


{'query': 'What is Harry James Potter birthday?',
 'no_ans_gap': -0.4726104736328125,
 'answers': [<Answer {'answer': '1980', 'type': 'extractive', 'score': 0.2529868185520172, 'context': "31 July is the 212th day of the year (213th in leap years). There are 153 days remaining until the end of the year.\n\n\n==Events==\n*Hogwarts School of Witchcraft and Wizardry expects to receive an owl confirming the receipt of the Hogwarts acceptance letter by non-Muggle-born students by this date.Harry James Potter was born on this day in 1980\n*1991: Rubeus Hagrid arrives at the Hut-on-the-Rock shortly after midnight and delivers Harry Potter his Hogwarts acceptance letter. Hagrid takes Harry on his first visit to Diagon Alley. Quirinus Quirrell breaks into Gringotts Wizarding Bank, but fails to steal the Philosopher's Stone as the vault containing it had already been emptied earlier in the day.\n*1992: The Dursleys have dinner with Mr and Mrs Mason. Dobby visits Harry, and warns him not to retur

In [6]:
qa.run(query='What are the four Hogwarts houses?',
       params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|██████████| 3/3 [00:27<00:00,  9.33s/ Batches]


{'query': 'What are the four Hogwarts houses?',
 'no_ans_gap': 14.121575355529785,
 'answers': [<Answer {'answer': 'Gryffindor, Hufflepuff, Ravenclaw or Slytherin', 'type': 'extractive', 'score': 0.9528752565383911, 'context': 'The four Hogwarts founders that were also the first Heads of Houses\nHead of House is the title for the Professors at Hogwarts School of Witchcraft and Wizardry who are responsible for the one of the four Houses: Gryffindor, Hufflepuff, Ravenclaw or Slytherin. They may serve concurrently as Deputy Headmaster but not as Headmaster.\n\n\n==Selection==\nProvided they were in the House while a student at Hogwarts, teachers at Hogwarts may apply for the role. They are most likely selected by the Head of the school.\nSeverus Snape was made Head of Slytherin at the remarkably young age of twenty-one, when he only just returned to Hogwarts. It may be that he was the only teacher who was in Slytherin, following the retirement of Horace Slughorn in 1981. When Slughorn ret

In [7]:
qa.run(query='When did Harry Potter first visit Hogsmeade?',
       params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|██████████| 2/2 [00:19<00:00,  9.96s/ Batches]


{'query': 'When did Harry Potter first visit Hogsmeade?',
 'no_ans_gap': 8.10150694847107,
 'answers': [<Answer {'answer': 'Christmas of 1997', 'type': 'extractive', 'score': 0.9403582215309143, 'context': "low was a village in the West Country of England. It was a small community, which centred on a village square with only a church, a post office, a pub, and a few retail shops. The residential streets were lined with quaint cottages, there was an area called Church Lane that lead up to the church.\nGodric's Hollow was inhabited by a number of notable wizarding families. The Dumbledore family and Bathilda Bagshot both resided in the village. It was perhaps most famous as where Harry Potter and his parents lived when he was a baby, where they were murdered, and Lord Voldemort met his first downfall. Thus, the village was notable as the place where Harry Potter became known as the 'Boy Who Lived'. Despite this, Harry did not visit until Christmas of 1997.\nAccording to ''A History of Ma

In [8]:


qa.run(query='What position does Harry play on the Gryffindor Quidditch team?',
       params={"Reader": {"top_k": 3}})

Inferencing Samples: 100%|██████████| 3/3 [00:36<00:00, 12.02s/ Batches]


{'query': 'What position does Harry play on the Gryffindor Quidditch team?',
 'no_ans_gap': 15.92770516872406,
 'answers': [<Answer {'answer': 'Captain', 'type': 'extractive', 'score': 0.9925570487976074, 'context': 'osing the person she thought would annoy Ron the most. He accepted, but spent most of the evening boasting about himself, to her irritation. He was very forward with her under the mistletoe, prompting Hermione to abandon him to seek out Harry Potter and Luna Lovegood and to avoid him for the rest of the party. He was very annoyed when he was unable to find her. This indicates that while McLaggen was romantically interested in Hermione, she found him to be a rude braggart.\n\n\n===Harry Potter===\nAlthough they were in the same House, Harry strongly disliked McLaggen. The two first met in 1996 when they were both questioned by Professor Slughorn. They met again when McLaggen tried out for the Gryffindor Quidditch team, of which Harry was the Captain. It seemed that McLaggen